Ahmed Ali 
22i-0825 E
Q2 Genetic Algorithm


In [ ]:
from dataclasses import dataclass
from typing import List

@dataclass
class Product:
    id: int
    name: str
    weight: float
    high_demand: int  # 0 or 1
    category: int     # e.g., 1 = Dairy, 2 = Grains
    perishable: int   # 0 or 1
    hazardous: int    # 0 or 1
    luxury: int       # 0 or 1
    promo: int        # 0 or 1
    complements: List[int]  # IDs of complementary products

@dataclass
class Shelf:
    id: int
    name: str
    capacity: float
    type: str  # "General", "Refrigerated", "Hazardous", "Checkout"
    height: str  # "Lower", "Eye-Level", "Upper"
    visibility: int  # 1 (low) to 3 (high)

# Example Data
products = [
    Product(0, "Milk", 5, 1, 1, 1, 0, 0, 0, []),
    Product(1, "Rice Bag", 10, 0, 2, 0, 0, 0, 0, []),
    Product(2, "Frozen Nuggets", 5, 0, 3, 1, 0, 0, 0, []),
    Product(3, "Cereal", 3, 1, 2, 0, 0, 0, 1, []),
    Product(4, "Pasta", 2, 0, 2, 0, 0, 0, 0, [5]),
    Product(5, "Pasta Sauce", 3, 0, 2, 0, 0, 0, 0, [4]),
    Product(6, "Detergent", 4, 0, 4, 0, 1, 0, 0, []),
    Product(7, "Glass Cleaner", 5, 0, 4, 0, 1, 0, 0, [])
]

shelves = [
    Shelf(0, "S1 - Checkout", 8, "Checkout", "Eye-Level", 3),
    Shelf(1, "S2 - Lower", 25, "General", "Lower", 1),
    Shelf(2, "S4 - Eye-Level", 15, "General", "Eye-Level", 2),
    Shelf(3, "S5 - General", 20, "General", "Upper", 1),
    Shelf(4, "R1 - Refrigerator", 20, "Refrigerated", "Eye-Level", 2),
    Shelf(5, "H1 - Hazardous", 10, "Hazardous", "Upper", 1)
]

def calculate_fitness(chromosome: List[int], products: List[Product], shelves: List[Shelf]) -> float:
    penalty = 0
    shelf_load = {s.id: [] for s in shelves}  # Track products per shelf

    # Assign products to shelves
    for i in range(len(chromosome)):
        shelf_id = chromosome[i]
        shelf_load[shelf_id].append(products[i])

    # Check constraints
    for shelf in shelves:
        items = shelf_load[shelf.id]
        total_weight = sum(p.weight for p in items)

        # 1. Shelf Capacity
        if total_weight > shelf.capacity:
            penalty += 10 * (total_weight - shelf.capacity)

        # 2. High-Demand Accessibility
        if shelf.height != "Eye-Level" and shelf.type != "Checkout":
            penalty += 5 * sum(p.high_demand for p in items)

        # 3. Category Segmentation
        categories = set(p.category for p in items)
        if len(categories) > 1 and shelf.type not in ["Checkout"]:
            penalty += 2 * (len(categories) - 1)

        # 4. Perishable Placement
        if shelf.type != "Refrigerated":
            penalty += 10 * sum(p.perishable for p in items)

        # 5. Hazardous Placement
        if shelf.type != "Hazardous" and any(p.hazardous for p in items):
            penalty += 10
        elif shelf.type == "Hazardous" and any(not p.hazardous for p in items):
            penalty += 5

        # 6. Cross-Selling (Reward)
        for p in items:
            if any(c in [prod.id for prod in items] for c in p.complements):
                penalty -= 1

        # 7. Restocking Efficiency
        if shelf.height != "Lower" and any(p.weight > 5 for p in items):
            penalty += 3

        # 9. Promotional Visibility
        if shelf.visibility < 2:
            penalty += 3 * sum(p.promo for p in items)

        # 10. Theft Prevention
        if shelf.visibility < 2 and any(p.luxury for p in items):
            penalty += 5

    # 8. Refrigeration Efficiency
    refrigerated_shelves = [s for s in shelves if s.type == "Refrigerated"]
    used_refrigs = set(chromosome[p.id] for p in products if p.perishable)
    if len(used_refrigs) > 1:
        penalty += 5 * (len(used_refrigs) - 1)

    return penalty


import random

def initialize_population(pop_size: int, num_products: int, num_shelves: int) -> List[List[int]]:
    return [[random.randint(0, num_shelves - 1) for _ in range(num_products)] for _ in range(pop_size)]

def tournament_selection(population: List[List[int]], fitnesses: List[float], k=3) -> List[int]:
    tournament = random.sample(list(zip(population, fitnesses)), k)
    return min(tournament, key=lambda x: x[1])[0]

def crossover(parent1: List[int], parent2: List[int]) -> List[int]:
    point = random.randint(1, len(parent1) - 1)
    return parent1[:point] + parent2[point:]

def mutate(chromosome: List[int], num_shelves: int, mutation_rate=0.01) -> List[int]:
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = random.randint(0, num_shelves - 1)
    return chromosome

def genetic_algorithm(products: List[Product], shelves: List[Shelf], pop_size=100, generations=1000):
    population = initialize_population(pop_size, len(products), len(shelves))
    for _ in range(generations):
        fitnesses = [calculate_fitness(chrom, products, shelves) for chrom in population]
        new_population = []
        for _ in range(pop_size // 2):
            parent1 = tournament_selection(population, fitnesses)
            parent2 = tournament_selection(population, fitnesses)
            child1 = crossover(parent1.copy(), parent2.copy())
            child2 = crossover(parent2.copy(), parent1.copy())
            new_population.extend([mutate(child1, len(shelves)), mutate(child2, len(shelves))])
        population = new_population
    fitnesses = [calculate_fitness(chrom, products, shelves) for chrom in population]
    best_chrom = population[fitnesses.index(min(fitnesses))]
    return best_chrom, min(fitnesses)

# Run GA
best_solution, best_fitness = genetic_algorithm(products, shelves)
print(f"Best Solution: {best_solution}, Fitness: {best_fitness}")



Best Solution: [4, 1, 4, 0, 3, 3, 5, 5], Fitness: 0


In [3]:
import pandas as pd

def export_to_excel(solution: List[int], products: List[Product], shelves: List[Shelf], filename="shelf_plan.xlsx"):
    data = {
        "Product ID": [p.id for p in products],
        "Product Name": [p.name for p in products],
        "Weight": [p.weight for p in products],
        "Shelf ID": solution,
        "Shelf Name": [shelves[sid].name for sid in solution]
    }
    df = pd.DataFrame(data)
    df.to_excel(filename, index=False)
    print(f"Solution exported to {filename}")

export_to_excel(best_solution, products, shelves)

Solution exported to shelf_plan.xlsx
